# Collaborative Filtering RS

In [1]:
# Analytical librairies 

%matplotlib notebook
import pandas as pd  #load csv files, merge(), corrwith() functions, etc

import numpy as np # filter Nan values


In [2]:

# Upload datasets as a movie dataframe which reflects the factorization matrix from the CFRS
MR_df = pd.read_csv('/media/mohinfo/MOH@/ml-latest-small/ratings.csv')

# Upload movies dataset to extract titles 
Movies = pd.read_csv('/media/mohinfo/MOH@/ml-latest-small/movies.csv')



In [14]:
Movies.sample(15)
#MR_df.head()

,movieId,title,genres
5656,27539,Undertaking Betty (Plots with a View) (2002),Comedy|Romance
950,1251,8 1/2 (8½) (1963),Drama|Fantasy
7456,81591,Black Swan (2010),Drama|Thriller
581,714,Dead Man (1995),Drama|Mystery|Western
1045,1357,Shine (1996),Drama|Romance
9,10,GoldenEye (1995),Action|Adventure|Thriller
3324,4499,Dirty Rotten Scoundrels (1988),Comedy
6097,42191,Luxo Jr. (1986),Animation|Children
9175,149011,He Never Died (2015),Comedy|Drama|Horror
7522,84374,No Strings Attached (2011),Comedy|Romance


In [4]:
# merging movies and rating datasets to join titles with users and their ratings based on movieID
Rat_Tit = pd.merge(MR_df, Movies, on='movieId')
Rat_Tit.head()


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [12]:
# statistical values
nums = pd.DataFrame(Rat_Tit.describe().transpose())
nums

,count,mean,std,min,25%,50%,75%,max
userId,100836.0,3.261276e+02,1.826185e+02,1.0,1.770000e+02,3.250000e+02,4.770000e+02,6.100000e+02
movieId,100836.0,1.943530e+04,3.553099e+04,1.0,1.199000e+03,2.991000e+03,8.122000e+03,1.936090e+05
rating,100836.0,3.501557e+00,1.042529e+00,0.5,3.000000e+00,3.500000e+00,4.000000e+00,5.000000e+00
timestamp,100836.0,1.205946e+09,2.162610e+08,828124615.0,1.019124e+09,1.186087e+09,1.435994e+09,1.537799e+09


In [5]:
# CFRS puts more interest on users' ratings, therefore, I'll implemet a depth view of
# the Rat_Tit dataset by highlighting ratings with movies titles

# The mean rating for each item can give a measure on the movies' prevalences
Rat_Tit.groupby('title')['rating'].mean().sort_values(ascending=False).head(10)


title
Karlson Returns (1970)                           5.0
Winter in Prostokvashino (1984)                  5.0
My Love (2006)                                   5.0
Sorority House Massacre II (1990)                5.0
Winnie the Pooh and the Day of Concern (1972)    5.0
Name: rating, dtype: float64

In [6]:
# Counter of ratings number for each movie
Rat_Tit.groupby('title')['rating'].count().sort_values(ascending=False).head()


title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: rating, dtype: int64

In [7]:
Rat = pd.DataFrame(Rat_Tit.groupby('title')['rating'].mean())
Rat['Num of ratings'] = pd.DataFrame(Rat_Tit.groupby('title')['rating'].count())

Rat.head()

,rating,Num of ratings
title,,
'71 (2014),4.0,1
'Hellboy': The Seeds of Creation (2004),4.0,1
'Round Midnight (1986),3.5,2
'Salem's Lot (2004),5.0,1
'Til There Was You (1997),4.0,2


In [8]:
# sort rating data
Rat.sort_values('Num of ratings', ascending= False).head()

,rating,Num of ratings
title,,
Forrest Gump (1994),4.164134,329
"Shawshank Redemption, The (1994)",4.429022,317
Pulp Fiction (1994),4.197068,307
"Silence of the Lambs, The (1991)",4.161290,279
"Matrix, The (1999)",4.192446,278


In [9]:
# Figure out a way to sort data according to the columns
MovMat = Rat_Tit.pivot_table(index= 'userId', columns= 'title', values='rating')
MovMat.head()
print(MovMat)

title   '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \
userId                                                        
1              NaN                                      NaN   
2              NaN                                      NaN   
3              NaN                                      NaN   
4              NaN                                      NaN   
5              NaN                                      NaN   
6              NaN                                      NaN   
7              NaN                                      NaN   
8              NaN                                      NaN   
9              NaN                                      NaN   
10             NaN                                      NaN   
11             NaN                                      NaN   
12             NaN                                      NaN   
13             NaN                                      NaN   
14             NaN                                     

In [10]:
# One important observation is that data is organized according to specific movies
# Therefore, we can define correlations, exp. between Black Swan (2010) & He Never Died (2015)
# as follows:

black_swan_ratings = MovMat['Black Swan (2010)']
He_Never_died_ratings = MovMat['He Never Died (2015)']
bs = black_swan_ratings.head() is not np.nan

# A correlation means at least one value which is not NaN, test done by "np.nan"

print(bs)


True


In [30]:
print(black_swan_ratings)

# 3.5 is a correlation value, where 'He never died' with most viewers -lengt: 610

userId
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     1.0
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
      ... 
581    NaN
582    NaN
583    NaN
584    NaN
585    NaN
586    NaN
587    NaN
588    NaN
589    NaN
590    4.0
591    NaN
592    NaN
593    NaN
594    NaN
595    NaN
596    NaN
597    NaN
598    NaN
599    4.0
600    NaN
601    NaN
602    NaN
603    NaN
604    NaN
605    NaN
606    NaN
607    NaN
608    NaN
609    NaN
610    3.5
Name: Black Swan (2010), Length: 610, dtype: float64


In [18]:
# Let's define similarities between movies

#similarTo_HeNeverdied = MovMat.corrwith(He_Never_died_ratings)
similarTo_BlackSwan = MovMat.corrwith(black_swan_ratings)

correlat_blackswan = pd.DataFrame(similarTo_BlackSwan, columns=['Correlations'])
correlat_blackswan.dropna(inplace= True)
correlat_blackswan.head()


,Correlations
title,
(500) Days of Summer (2009),0.300676
*batteries not included (1987),1.000000
10 Cent Pistol (2015),1.000000
10 Cloverfield Lane (2016),0.010478
10 Things I Hate About You (1999),0.049153


In [25]:
# Best similarity (recommendation) for a given movie is the one(s) which is highly rated
# THe decision threshold to pick the right recommended movies based on a pre-selection is 
# let to users' rigor. In the following test, an acceptable recommended movie rating should be 
# beyond 70

correlat_blackswan.sort_values('Correlations', ascending= False).head(10)
correlat_blackswan = correlat_blackswan.join(Rat['Num of ratings'])

correlat_blackswan.head()
correlat_blackswan = correlat_blackswan[correlat_blackswan['Num of ratings'] 
                                        > 70].sort_values('Correlations', ascending= False)

print(correlat_blackswan)

                                                    Correlations  \
title                                                              
Birdcage, The (1996)                                    0.844951   
Leaving Las Vegas (1995)                                0.828079   
Nightmare Before Christmas, The (1993)                  0.788671   
Star Trek: First Contact (1996)                         0.740744   
Mars Attacks! (1996)                                    0.735321   
Outbreak (1995)                                         0.727860   
Gattaca (1997)                                          0.697845   
Beetlejuice (1988)                                      0.697700   
Interview with the Vampire: The Vampire Chronic...      0.688297   
Apocalypse Now (1979)                                   0.677455   
Crouching Tiger, Hidden Dragon (Wo hu cang long...      0.660488   
American Beauty (1999)                                  0.652052   
Prestige, The (2006)                            